In [1]:
import os
import sys
import numpy as np
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import pixel as pix
import utilities as util
import tile
from unet_model import unet_model


Using TensorFlow backend.


In [2]:
from sklearn.utils import class_weight

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"]="2";

In [4]:
landsat, sentinel, dem, label = util.load_data()
tile_size = 64
num_classes = util.get_class_count()
pixel_gen = pix.pixel_gen(landsat, sentinel, dem, label, tile_size, num_classes)
tile_gen = tile.tile_gen(landsat, sentinel, dem, label, tile_size, num_classes)

In [5]:
pixel_to_balance = pixel_gen.gen_pixels(1000000, balanced=False)
pixels = pixel_gen.gen_pixels(40000, balanced=False)

pixels generated 557856
pixels generated 22446


In [6]:
balance = pixel_gen.calculate_balance(pixel_to_balance)
pixel_gen.print_balance()

Water:52670.0
Snow/Ice:0.0
Open Space Developed:16427.0
Low Intensity Developed:0.0
Medium Intensity Developed:10359.0
High Intensity Developed:0.0
Barren Land:476.0
Deciduous Forest:3773.0
Evergreen Forest:82464.0
Mixed Forest:10295.0
Scrub/Shrub:15632.0
Grassland / Herbaceous:8509.0
Pasture/Hay:2900.0
Cultivated Land:100965.0
Woody Wetland:118664.0
Emergent Herbaceous Wetlands:6170.0


In [16]:
train_px, test_px, val_px = pixel_gen.train_val_test_split(pixel_to_balance, 0.7, 0.7)

train:390499 val:117149 test:85353


In [8]:
classes = dict(zip(np.arange(num_classes), balance))
y_true = []
for index, amount in classes.items():
    y_true = np.concatenate((y_true, [index]*int(amount)))

In [9]:
weights_list = class_weight.compute_class_weight('balanced', np.unique(y_true), y_true)
weights_dict = dict(zip(np.unique(y_true).astype(int), weights_list))
for i in range(num_classes):
    if i not in weights_dict.keys():
        weights_dict[i] = 0

In [10]:
c_weights = list(weights_dict.values())

In [11]:
batch_size = 25
epochs = 30

In [17]:
model = unet_model(c_weights, n_classes=num_classes, im_sz=tile_size, n_channels=10, n_filters_start=64, growth_factor=2)

In [18]:
model.fit_generator(generator=tile_gen.tile_generator(train_px, batch_size, fcn=True), 
                    steps_per_epoch=len(train_px) // batch_size, epochs=epochs, verbose=1,
                    validation_data=tile_gen.tile_generator(val_px, batch_size, fcn=True),
                    validation_steps=len(val_px) // batch_size)

Epoch 1/30
  113/15619 [..............................] - ETA: 2:56:59 - loss: 6.5366 - acc: 0.0041

KeyboardInterrupt: 